In [1]:
import os
import time
import torch
from torch import nn
from torchvision import transforms
import torchvision
import matplotlib.pyplot as plt
import datasets
import pytorch_lightning as pl
from torch import nn
from torch.nn import functional as F
from pl_bolts.models.autoencoders.components import (
    resnet18_decoder,
    resnet18_encoder,
)




#torch.set_default_tensor_type('torch.cuda.FloatTensor')
tensor_transform = transforms.ToTensor()


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print('Device:',device)

D:\Anaconda\envs\pytorch\lib\site-packages\pl_bolts\models\self_supervised\amdim\amdim_module.py:35: UnderReviewWarning: The feature generate_power_seq is currently marked under review. The compatibility with other Lightning projects is not guaranteed and API may change at any time. The API and functionality may change without warning in future releases. More details: https://lightning-bolts.readthedocs.io/en/latest/stability.html
  "lr_options": generate_power_seq(LEARNING_RATE_CIFAR, 11),
D:\Anaconda\envs\pytorch\lib\site-packages\pl_bolts\models\self_supervised\amdim\amdim_module.py:93: UnderReviewWarning: The feature FeatureMapContrastiveTask is currently marked under review. The compatibility with other Lightning projects is not guaranteed and API may change at any time. The API and functionality may change without warning in future releases. More details: https://lightning-bolts.readthedocs.io/en/latest/stability.html
  contrastive_task: Union[FeatureMapContrastiveTask] = Feature

Device: cuda


In [ ]:
#Helper functions
def plot():
    f, axarr = plt.subplots(2)

    for i, item in enumerate(image):
    # Reshape the array for plotting
        item = item.reshape(-1, 28, 28)
        axarr[0].imshow(item[0].cpu())

    for i, item in enumerate(reconstructed):
        item = item.reshape(-1, 28, 28).cpu()
        item = item.detach().numpy()
        axarr[1].imshow(item[0])
        
        
        
def showExample():
    for image, _ in example_loader:
        f, axarr = plt.subplots(2)
        image = image.reshape(-1,28*28).to(device)

        model.to(device)
        recon = model(image)

        image = image.reshape(-1, 28, 28)
        axarr[0].imshow(image[0].cpu())


        recon = recon.reshape(-1, 28, 28).to('cpu')
        axarr[1].imshow(recon[0].detach().numpy())

        break

def add_noise(inputs,variance):
    noise = torch.randn_like(inputs)
    return inputs + variance*noise

        
        
        
        

### KL Divergence intuition
How we are minimizing KL

In [ ]:
def kl_divergence(self,z,mu,std):
    p = torch.distributions.Normal(torch.zeros_like(mu), torch.ones_like(std))
    q = torch.distributions.Normal(mu, std)
    
    log_pz = p.log_prob(z)
    log_qzx = q.log_prob(z)
    
    kl = (log_qxz - log_pz)
    
    #sum over last dim to go from single dim distr. to multi-dim
    kl = kl.sum(-1)
    return kl


    

In [ ]:
input_size = 28*28  #784
hidden_size = 128
code_size = 32


class VAE(nn.Module):
    def __init__(self):
        super().__init__()
        
        #Encoder 
        self.encoder = nn.Sequential(
            nn.Linear(input_size,hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size,code_size)
        )
        
        #Decoder
        self.decoder = nn.Sequential(
            nn.Linear(code_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, input_size),
            nn.Sigmoid()
        )
        self.model = nn.Sequential(
        
        )    
        
    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded
    
        
        

In [ ]:
class VAE(pl.LightningModule):
    def __init__(self, enc_out_dim=512, latent_dim=256, input_height=32):
        super().__init__()
        
        
        self.save_hyperparameters()
        
        
        
        self.encoder = resnet18_encoder(False,False)
        self.decoder = resnet18_decoder(latent_dim = latent_dim,
                                        input_height = input_height,
                                        first_conv=False,
                                        maxpool1=False
                                        
                                        )
        
        self.fc_mu = nn.Linear(enc_out_dim,latent_dim)
        self.fc_var = nn.Linear(enc_out_dim,latent_dim)
        
        #for gaussian distribution
        self.log_scale = nn.Parameter(torch.Tensor([0.0]))
        
        def configure_optimizers(self):
            return torch.optim.Adam(self.parameters(),lr=1e-4)
        
        
        def gaussian_likelihood(self,x_hat,logscale,x):
            scale = torch.exp(logscale)
            mean = x_hat
            dist = torch.distributions.Normal(mean,scale)
            
            
            #prob of seing z under p(x|z)
            log_pxz = dist.log_prob(x)
            return log_pxz.sun(dim=(1,2,3))
        
        def kl_divergence(self,z,mu,std):
            #define probabilities
            p = torch.distributions.Normal(torch.zeros_like(mu),
                                           torch.ones_like(std))
            
            #2. get probabilbities from equation
            log_qzx = q.log_prob(z)
            log_pz = p.log_prob(z)
            
            
            #kl
            kl = (log_qzx- log_pz)
            kl = kl.sum(-1)
            return kl
        
        
        
        def training_step(self,batch,batch_idx):
            x, = batch
            
            #encode x to get mu and std
            x_encoded = self.encoder(x)
            mu,log_var = self.fc_mu(x_encoded),self.fc_var(x_encoded)
            
            #sample z from q
            std = torch.exp(log_var / 2)
            q = torch.distributions.Normal(mu,std)
            z = q.rsample()
            
            
            #decoded
            x_hat = self.decoder(z)
            
            #reconstruction loss
            recon_loss = self.gaussian_likelihood(x_hat,self.log_scale,x)
            
            #kl
            kl = self.kl_divergence(z,mu,std)
            
            #elbo
            elbo = (kl-recon_loss) 
            elbo = elbo.mean()
            
            self.log_dict({
            'elbo': elbo,
            'kl': kl.mean(),
            'recon_loss': recon_loss.mean(),
            'reconstruction': recon_loss.mean(),
            'kl': kl.mean(),
        })
        
            return elbo

First distribution:  
**q(z|x)**  
We generate mu, std from the encoder
    

In [ ]:
vae  = VAE()

#image from cifar-10(3 channels, 32*32)
x = torch.rand(1,3,32,32)
print('Image Shape {}'.format(x.shape))

#Encode x to get mu and variance parameters
x_encoded = vae.encoder(x)
mu, log_var = vae.fc_mu(x_encoded),vae.fc_var(x_encoded)

print('Encoded Image Shape {}'.format(x_encoded.shape))
print('μ {}'.format(mu.shape))
print('log_var {}'.format(log_var.shape))

# SAMPLE Z from Q(Z|x)
std = torch.exp(log_var / 2)
q = torch.distributions.Normal(mu, std)
z = q.rsample()

print('z shape:', z.shape)



The second distribution:  
**p(z)**   
is the prior which we will fix to specific location(0,1)  
By doing this, KL Willl force q(z|x) to move close to p

At start, distributions will look like this:
![](markdown/distributions.jpg)



And over time, the q will mmove close to p, as it has learnable parameters

![](markdown/distributions-over-time.jpg)

In [ ]:
zero = torch.zeros_like(mu)
one = torch.ones_like(std)
p = torch.distributions.Normal(zero,one)

The third distribution:  
**p(x|z)**  
(or reconstruction) will be used to measure the probability of seeing the image give that z was sampled

Note: x_hat is **not** an image. They are parameters for a distribution  
When used with MNIST dataset, they can form an image, but for colored images this isn't true 

In [ ]:
x_hat = vae.decoder(z)
print(x_hat.shape)


Lastly, we use x to parameterize a likehood distribution so that we can measure the probability of an input uder the (very highly dimensional) distribution

In [ ]:
log_scale = nn.Parameter(torch.Tensor([0.0]))
scale = torch.exp(log_scale)
dist = torch.distributions.Normal(x_hat,scale)
log_pxz = dist.log_prob(x)
print(log_pxz.shape)


#sum across all channels, pixels
log_pxz = log_pxz.sum(dim=(1,2,3))
print(log_pxz.shape)
print('recon Loss {}'.format(log_pxz.item()))

recon_loss = log_pxz

In [ ]:
from pl_bolts.datamodules import CIFAR10DataModule, ImagenetDataModule

# you can use ImagenetDataModule if you want
# imagenet = ImagenetDataModule('.')

# for this tutorial we'll use cifar10
cifar_10 = CIFAR10DataModule('.')

pl.seed_everything(1234)

vae = VAE()
trainer = pl.Trainer(gpus=1, max_epochs=30, progress_bar_refresh_rate=10)
trainer.fit(vae, cifar_10)



In [ ]:

from matplotlib.pyplot import imshow, figure
import numpy as np
from torchvision.utils import make_grid
from pl_bolts.transforms.dataset_normalizations import cifar10_normalization
figure(figsize=(8, 3), dpi=300)

# Z COMES FROM NORMAL(0, 1)
num_preds = 16
p = torch.distributions.Normal(torch.zeros_like(mu), torch.ones_like(std))
z = p.rsample((num_preds,))

# SAMPLE IMAGES
with torch.no_grad():
    pred = vae.decoder(z.to(vae.device)).cpu()

# UNDO DATA NORMALIZATION
normalize = cifar10_normalization()
mean, std = np.array(normalize.mean), np.array(normalize.std)
img = make_grid(pred).permute(1, 2, 0).numpy() * std + mean

# PLOT IMAGES
imshow(img);